In [2]:
# CONEXIÓN A BASE DE DATOS 
import pandas as pd
from sqlalchemy import create_engine


db_config = {
    'user': 'practicum_student',
    'pwd': 's65BlTKV3faNIGhmvJVzOqhs',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],

    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

print("✅ CONEXIÓN ESTABLECIDA A BASE DE DATOS SQL")
print("🔍 Base de datos: data-analyst-final-project-db")
print("📊 Iniciando análisis d
e base de datos de libros...")

✅ CONEXIÓN ESTABLECIDA A BASE DE DATOS SQL
🔍 Base de datos: data-analyst-final-project-db
📊 Iniciando análisis de base de datos de libros...


# 📚 ANÁLISIS SQL - BASE DE DATOS DE LIBROS

## 🎯 OBJETIVOS DEL ESTUDIO
**Contexto:** Startup desarrollando aplicación para amantes de libros durante pandemia

**Propósito:** Generar propuesta de valor para nuevo producto

**Preguntas a responder:**
1. ¿Cuántos libros modernos (post-2000) hay disponibles?
2. ¿Qué libros tienen más reseñas y mejores calificaciones?
3. ¿Qué editorial domina el mercado de libros extensos?
4. ¿Qué autores son los mejor calificados?
5. ¿Cómo se comportan los usuarios más activos?

In [3]:
# EXPLORACIÓN DE TABLAS - PRIMERAS FILAS
print("🔍 EXPLORANDO ESTRUCTURA DE TABLAS")
print("=" * 50)

# Lista de todas las tablas disponibles
tablas = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for tabla in tablas:
    print(f"\n📊 TABLA: {tabla.upper()}")
    query = f"SELECT * FROM {tabla} LIMIT 3;"
    try:
        df = pd.io.sql.read_sql(query, con=engine)
        print(f"📏 Forma: {df.shape} (filas, columnas)")
        print(f"🎯 Columnas: {list(df.columns)}")
        print("📋 Primeras filas:")
        display(df)
        print("-" * 40)
    except Exception as e:
        print(f"❌ Error al acceder a la tabla {tabla}: {e}")

🔍 EXPLORANDO ESTRUCTURA DE TABLAS

📊 TABLA: BOOKS
📏 Forma: (3, 6) (filas, columnas)
🎯 Columnas: ['book_id', 'author_id', 'title', 'num_pages', 'publication_date', 'publisher_id']
📋 Primeras filas:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


----------------------------------------

📊 TABLA: AUTHORS
📏 Forma: (3, 2) (filas, columnas)
🎯 Columnas: ['author_id', 'author']
📋 Primeras filas:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


----------------------------------------

📊 TABLA: PUBLISHERS
📏 Forma: (3, 2) (filas, columnas)
🎯 Columnas: ['publisher_id', 'publisher']
📋 Primeras filas:


...

----------------------------------------

📊 TABLA: RATINGS
📏 Forma: (3, 4) (filas, columnas)
🎯 Columnas: ['rating_id', 'book_id', 'username', 'rating']
📋 Primeras filas:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


----------------------------------------

📊 TABLA: REVIEWS
📏 Forma: (3, 4) (filas, columnas)
🎯 Columnas: ['review_id', 'book_id', 'username', 'text']
📋 Primeras filas:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


----------------------------------------


In [4]:
# CONSULTA 1: Libros publicados después del 1 de enero de 2000
print("1️⃣ CONSULTA 1: Libros publicados después del 1 de enero de 2000")
print("=" * 60)

query_1 = """
SELECT COUNT(*) as total_libros
FROM books
WHERE publication_date > '2000-01-01';
"""

try:
    resultado_1 = pd.io.sql.read_sql(query_1, con=engine)
    total_libros = resultado_1['total_libros'][0]
    print(f"📚 RESULTADO: {total_libros} libros publicados después del 1 de enero de 2000")
    display(resultado_1)
    
    # Conclusión simple
    print(f"\n💡 CONCLUSIÓN: Existen {total_libros} libros modernos para potencial inclusión en la app")
    
except Exception as e:
    print(f"❌ Error en consulta 1: {e}")

1️⃣ CONSULTA 1: Libros publicados después del 1 de enero de 2000
📚 RESULTADO: 819 libros publicados después del 1 de enero de 2000


,total_libros
0,819



💡 CONCLUSIÓN: Existen 819 libros modernos para potencial inclusión en la app


In [5]:
# CONSULTA 2: Número de reseñas y calificación promedio por libro
print("2️⃣ CONSULTA 2: Reseñas y calificación promedio por libro")
print("=" * 60)

query_2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) as numero_resenas,
    ROUND(AVG(rt.rating), 2) as calificacion_promedio,
    COUNT(DISTINCT rt.rating_id) as total_calificaciones
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
HAVING COUNT(DISTINCT rt.rating_id) > 0
ORDER BY numero_resenas DESC, calificacion_promedio DESC
LIMIT 10;
"""

try:
    resultado_2 = pd.io.sql.read_sql(query_2, con=engine)
    print("📖 TOP 10 libros con más reseñas y mejores calificaciones:")
    display(resultado_2)
    
    # Conclusión
    mejor_libro = resultado_2['title'].iloc[0]
    mejor_calificacion = resultado_2['calificacion_promedio'].iloc[0]
    print(f"\n💡 CONCLUSIÓN: '{mejor_libro}' es el libro más reseñado con calificación {mejor_calificacion}/5")
    
except Exception as e:
    print(f"❌ Error en consulta 2: {e}")

2️⃣ CONSULTA 2: Reseñas y calificación promedio por libro
📖 TOP 10 libros con más reseñas y mejores calificaciones:


,book_id,title,numero_resenas,calificacion_promedio,total_calificaciones
0,948,Twilight (Twilight #1),7,3.66,160
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41,82
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29,80
3,656,The Book Thief,6,4.26,53
4,734,The Glass Castle,6,4.21,29
5,497,Outlander (Outlander #1),6,4.13,24
6,750,The Hobbit or There and Back Again,6,4.13,88
7,695,The Curious Incident of the Dog in the Night-Time,6,4.08,37
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.08,62
9,963,Water for Elephants,6,3.98,44



💡 CONCLUSIÓN: 'Twilight (Twilight  #1)' es el libro más reseñado con calificación 3.66/5


In [6]:
# CONSULTA 3: Editorial con mayor número de libros >50 páginas
print("3️⃣ CONSULTA 3: Editorial con más libros de más de 50 páginas")
print("=" * 60)

query_3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) as total_libros
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_libros DESC
LIMIT 5;
"""

try:
    resultado_3 = pd.io.sql.read_sql(query_3, con=engine)
    print("🏢 TOP 5 editoriales con más libros extensos (>50 páginas):")
    display(resultado_3)
    
    # Conclusión
    editorial_lider = resultado_3['publisher'].iloc[0]
    total_libros = resultado_3['total_libros'].iloc[0]
    print(f"\n💡 CONCLUSIÓN: '{editorial_lider}' domina con {total_libros} libros extensos - socio potencial clave")
    
except Exception as e:
    print(f"❌ Error en consulta 3: {e}")

3️⃣ CONSULTA 3: Editorial con más libros de más de 50 páginas
🏢 TOP 5 editoriales con más libros extensos (>50 páginas):


,publisher,total_libros
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19



💡 CONCLUSIÓN: 'Penguin Books' domina con 42 libros extensos - socio potencial clave


In [7]:
# CONSULTA 4: Autor con mejor calificación promedio (mínimo 50 ratings)
print("4️⃣ CONSULTA 4: Autor con mejor calificación promedio")
print("=" * 60)

query_4 = """
SELECT 
    a.author,
    b.title,
    ROUND(AVG(r.rating), 2) as calificacion_promedio,
    COUNT(r.rating_id) as total_calificaciones
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
GROUP BY a.author, b.title
HAVING COUNT(r.rating_id) >= 50
ORDER BY calificacion_promedio DESC
LIMIT 5;
"""

try:
    resultado_4 = pd.io.sql.read_sql(query_4, con=engine)
    print("👨‍💼 TOP 5 autores mejor calificados (mínimo 50 ratings por libro):")
    display(resultado_4)
    
    if len(resultado_4) > 0:
        mejor_autor = resultado_4['author'].iloc[0]
        mejor_calificacion = resultado_4['calificacion_promedio'].iloc[0]
        print(f"\n💡 CONCLUSIÓN: '{mejor_autor}' es el autor mejor calificado ({mejor_calificacion}/5) - ideal para promociones")
    else:
        print("\n💡 CONCLUSIÓN: No hay autores con mínimo 50 calificaciones")
        
except Exception as e:
    print(f"❌ Error en consulta 4: {e}")

4️⃣ CONSULTA 4: Autor con mejor calificación promedio
👨‍💼 TOP 5 autores mejor calificados (mínimo 50 ratings por libro):


,author,title,calificacion_promedio,total_calificaciones
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.41,82
1,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,4.39,74
2,J.K. Rowling/Mary GrandPré,Harry Potter and the Chamber of Secrets (Harry...,4.29,80
3,Markus Zusak/Cao Xuân Việt Khương,The Book Thief,4.26,53
4,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,4.25,73



💡 CONCLUSIÓN: 'J.K. Rowling/Mary GrandPré' es el autor mejor calificado (4.41/5) - ideal para promociones


In [8]:
# CONSULTA 5: Promedio de reseñas entre usuarios que calificaron >50 libros
print("5️⃣ CONSULTA 5: Promedio de reseñas entre usuarios activos")
print("=" * 60)

query_5 = """
SELECT 
    ROUND(AVG(total_resenas), 2) as promedio_resenas_por_usuario
FROM (
    SELECT 
        username,
        COUNT(DISTINCT review_id) as total_resenas
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(DISTINCT book_id) > 50
    )
    GROUP BY username
) as usuarios_activos;
"""

try:
    resultado_5 = pd.io.sql.read_sql(query_5, con=engine)
    promedio_resenas = resultado_5['promedio_resenas_por_usuario'].iloc[0]
    print("📝 Comportamiento de usuarios que calificaron más de 50 libros:")
    display(resultado_5)
    
    print(f"\n💡 CONCLUSIÓN: Los usuarios más activos escriben en promedio {promedio_resenas} reseñas - valioso para programa de fidelización")
    
except Exception as e:
    print(f"❌ Error en consulta 5: {e}")

5️⃣ CONSULTA 5: Promedio de reseñas entre usuarios activos
📝 Comportamiento de usuarios que calificaron más de 50 libros:


,promedio_resenas_por_usuario
0,24.33



💡 CONCLUSIÓN: Los usuarios más activos escriben en promedio 24.33 reseñas - valioso para programa de fidelización


# 🎯 CONCLUSIONES FINALES - ANÁLISIS SQL

## 📊 RESUMEN DE HALLazGOS

### 1. 📚 Catálogo Moderno
- **X libros** publicados después del 2000
- Oportunidad de enfocarse en contenido contemporáneo

### 2. 🏢 Liderazgo Editorial  
- **Editorial X** domina con Y libros extensos
- Socio estratégico potencial para acuerdos de contenido

### 3. 👨‍💼 Autores Destacados
- **Autor Z** con mejor calificación promedio
- Ideal para contenido promocional y recomendaciones

### 4. 📈 Comportamiento de Usuarios
- Usuarios activos generan **N reseñas** en promedio
- Base sólida para programa de reseñistas

## 💡 RECOMENDACIONES PARA NUEVO PRODUCTO

1. **Enfoque en modernidad** - Destacar libros post-2000
2. **Alianzas estratégicas** - Colaborar con editorial líder
3. **Promoción de autores** - Featured sections para autores mejor calificados
4. **Programa de fidelización** - Incentivar a usuarios activos
5. **Sistema de recomendaciones** - Basado en patrones de calificación